In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [3]:
# Config
STREAMED_DATA_PATH = "/content/drive/MyDrive/SmartFactory/Results/Week5/StreamBatch"
ROLLING_WINDOW = 5
ROC_THRESHOLD = {
    "vibration": 1.0,
    "temperature": 15.0,
    "rpm": 500,
    "voltage": 15,
    "pressure": 3.0
}


In [4]:
sns.set(style="whitegrid")

In [5]:
# Detect anomalies
def detect_anomalies(df):
    anomalies = []

    for i in range(1, len(df)):
        row = df.iloc[i]
        prev = df.iloc[i - 1]
        anomaly_flags = {}

        for feature in ROC_THRESHOLD.keys():
            roc = abs(row[feature] - prev[feature])
            if roc > ROC_THRESHOLD[feature]:
                anomaly_flags[feature] = f"Δ {roc:.2f} > {ROC_THRESHOLD[feature]}"

        if anomaly_flags:
            anomaly_entry = row.to_dict()
            anomaly_entry["timestamp"] = row["timestamp"].isoformat()  # Convert timestamp to string
            anomaly_entry["anomaly_details"] = anomaly_flags
            anomalies.append(anomaly_entry)

    return anomalies

In [6]:
# Visualize anomalies
def visualize_anomalies(df, anomalies, batch_name):
    if not anomalies:
        return

    anomaly_df = pd.DataFrame(anomalies)
    anomaly_df["timestamp"] = pd.to_datetime(anomaly_df["timestamp"])
    df["timestamp"] = pd.to_datetime(df["timestamp"])

    metrics = ["temperature", "vibration", "rpm", "voltage", "pressure"]

    for metric in metrics:
        plt.figure(figsize=(10, 4))
        plt.plot(df["timestamp"], df[metric], label=f"{metric} (normal)", color='blue')
        if metric in anomaly_df.columns:
            plt.scatter(anomaly_df["timestamp"], anomaly_df[metric], color='red', label="Anomaly", s=50, zorder=5)
        plt.title(f"{metric.capitalize()} Trend with Anomalies – {batch_name}")
        plt.xlabel("Timestamp")
        plt.ylabel(metric.capitalize())
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join("/content/drive/MyDrive/SmartFactory/Results/Week6/AnomalyPlots", f"{batch_name}_{metric}_anomaly_plot.png"))
        plt.close()

In [7]:
# Monitoring pipeline
def monitor_batches():
    all_files = sorted(f for f in os.listdir(STREAMED_DATA_PATH) if f.startswith("stream_batch"))
    rolling_df = pd.DataFrame()
    total_anomalies = []

    for file in all_files:
        file_path = os.path.join(STREAMED_DATA_PATH, file)
        with open(file_path, 'r') as f:
            batch = json.load(f)

        df = pd.DataFrame(batch)
        df["timestamp"] = pd.to_datetime(df["timestamp"])

        # Update rolling window
        rolling_df = pd.concat([rolling_df, df], ignore_index=True)
        if len(rolling_df) > ROLLING_WINDOW + 1:
            rolling_df = rolling_df.iloc[-(ROLLING_WINDOW + 1):]

        anomalies = detect_anomalies(rolling_df)
        total_anomalies.extend(anomalies)

        print(f"[MONITOR] Processed {file} | Anomalies Detected: {len(anomalies)}")

        # Save anomalies
        if anomalies:
            out_file = os.path.join("/content/drive/MyDrive/SmartFactory/Results/Week6/Anomalies", f"anomalies_{file}")
            with open(out_file, 'w') as fout:
                json.dump(anomalies, fout, indent=2)

            # Visualize and save plots
            visualize_anomalies(rolling_df, anomalies, file.split(".")[0])

    # Save summary
    with open(os.path.join("/content/drive/MyDrive/SmartFactory/Results/Week6", "anomaly_summary.json"), 'w') as final_report:
        json.dump(total_anomalies, final_report, indent=2)

    print(f"Monitoring complete.")
    print(f"Total anomalies detected: {len(total_anomalies)}")
    print(f"Plots saved")

monitor_batches()

[MONITOR] Processed stream_batch_0.json | Anomalies Detected: 5
[MONITOR] Processed stream_batch_1.json | Anomalies Detected: 3
[MONITOR] Processed stream_batch_10.json | Anomalies Detected: 5
[MONITOR] Processed stream_batch_100.json | Anomalies Detected: 3
[MONITOR] Processed stream_batch_101.json | Anomalies Detected: 5
[MONITOR] Processed stream_batch_102.json | Anomalies Detected: 4
[MONITOR] Processed stream_batch_103.json | Anomalies Detected: 5
[MONITOR] Processed stream_batch_104.json | Anomalies Detected: 3
[MONITOR] Processed stream_batch_105.json | Anomalies Detected: 5
[MONITOR] Processed stream_batch_106.json | Anomalies Detected: 4
[MONITOR] Processed stream_batch_107.json | Anomalies Detected: 3
[MONITOR] Processed stream_batch_108.json | Anomalies Detected: 3
[MONITOR] Processed stream_batch_109.json | Anomalies Detected: 4
[MONITOR] Processed stream_batch_11.json | Anomalies Detected: 4
[MONITOR] Processed stream_batch_110.json | Anomalies Detected: 4
[MONITOR] Proces